# Plan of Attack
1. Create sample out of imdb data set
    * Preprocess for rare words (done [handled by load_data])
    * Pad all entries to 500 words (done [keras.preprocessing])
    * Simple randomly select 10% (done [random.shuffle zip trick])
    * Use bcolz to save off what we did (light weight data so not doing)
2. Create simple simple model to exercise functional api and verify stuff works
    * Grok the embedding and create simple functional layer of 1D convolution (done)
3. Bring in pretrained model and verify that we get similar performance
4. Attempt to beat jeremy's score (.90 val ac)
    * Can we preprocess words similar to how glove did it?
    * Deeper network
    * Number of words as a side channel using functional api?
    * 2d convolutions?

In [84]:
import os,sys,random, shutil
import numpy as np
#vgg is held in a subdirectory
sys.path.append('kernel')
import vgg16; reload(vgg16)
from vgg16 import Vgg16
import keras as k
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, BatchNormalization, Convolution1D
from keras.layers import Convolution2D, MaxPooling2D,Flatten,Dropout
from keras.layers import Input, Embedding
from keras.preprocessing import image,sequence
from keras.optimizers import SGD, RMSprop, Adam
#import bcolz
import utils
import random

## Create Sample out of IMDB data set

In [2]:
from keras.datasets import imdb

In [7]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = 5000)

In [9]:
type(x_train)

numpy.ndarray

In [54]:
padded_x_train = sequence.pad_sequences(x_train, maxlen=500)
padded_x_test = sequence.pad_sequences(x_test, maxlen=500)

In [56]:
y_train[0]

1

In [67]:
c = zip(padded_x_train, y_train)
random.shuffle(c)
x_train, y_train = zip(*c)

In [72]:
x_train = list(x_train)
y_train = list(y_train)

In [78]:
x_val = x_train[-2500:]
y_val = y_train[-2500:]
x_train = x_train[:len(x_train)-2500]
y_train = y_train[:len(y_train)-2500]
x_t = np.asarray(x_train)
y_t = np.asarray(y_train)
x_v = np.asarray(x_val)
y_v = np.asarray(y_val)
x_test = np.asarray(padded_x_test)
y_test = np.asarray(y_test)

## Create Simple Model 

In [109]:
# We start with an embedding layer with 32 random weights
main_input = Input(shape=(500,), dtype='int32' ,name='main_input')

embedding = Embedding(input_dim=5000, output_dim=32,input_length=500)(main_input)
c = Convolution1D(64, 5)(embedding)
f = Flatten()(c)
d = Dense(500, activation='relu')(f)
d = Dropout(.1)(d)
d = Dense(250, activation='relu')(d)
main_output = Dense(1, activation='sigmoid', name='main_output')(d)
model = Model(inputs=[main_input], outputs=[main_output])

In [110]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 500)               0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 496, 64)           10304     
_________________________________________________________________
flatten_5 (Flatten)          (None, 31744)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 500)               15872500  
_________________________________________________________________
dropout_2 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 250)               125250    
__________

In [111]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])


In [113]:
model.fit(x_t,y_t, batch_size=32, epochs=1, validation_data=(x_v,y_v))

Train on 22500 samples, validate on 2500 samples
Epoch 1/1
22500/22500 [==============================] - 538s - loss: 0.2249 - acc: 0.9097 - val_loss: 0.2986 - val_acc: 0.8792


In [102]:
x_t.shape

(22500, 500)

In [114]:
model.evaluate(x_test, y_test)

25000/25000 [==============================] - 121s   


[0.29590418436050414, 0.87612000000000001]

## PreTrained Model using Glove Vectors